In [54]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller #used to auto-install chromedriver if current version of chrome is different
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pprint
from bs4 import BeautifulSoup
import re

chromedriver_autoinstaller.install()  
driver = webdriver.Chrome()
url = "https://twitter.com/explore"
driver.get(url)

"""I COULD USE A REAL TWITTER ACCOUNT JUST FOR TEST BUT DUE TO A LOT OF TESTS IN ORDER TO FILTER THE DATA CORRECTLY, 
SOMETIMES I NEED TO PUT PHONE NUMBER IN ORDER TO VERIFY MY IDENTITY. SO, I DIRECTLY USED EXPLORE PAGE OF TWITTER""" 


# time.sleep(2)
# sig_but = driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div[1]/div[1]/div/div[3]/div[5]/a')
# time.sleep(1)
# sig_but.click()

# time.sleep(1)
# username = driver.find_element_by_xpath('//input[@name="text"]')
# time.sleep(1)
# username.send_keys('RAtestuser')
# time.sleep(1)
# username.send_keys(Keys.RETURN)

# time.sleep(3)
# password = driver.find_element_by_xpath('//input[@name="password"]')
# time.sleep(1)
# password.send_keys('Reatest1.')
# time.sleep(1)
# password.send_keys(Keys.RETURN)

#write in search
time.sleep(1)
input_search = driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[1]/div[1]/div/div/div/div/div[1]/div[2]/div/div/div/form/div[1]/div/div/label/div[2]/div/input')
input_search.send_keys("request for startup")
input_search.send_keys(Keys.ENTER)
time.sleep(2)

#latest 
driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[1]/div[2]/nav/div/div[2]/div/div[2]/a').click()


In [55]:
#tweets = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')

In [56]:
def get_tweet_data(tweet):
    username = tweet.find_element_by_xpath('.//span').text
    try:
        handle = tweet.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    try:
        postdate = tweet.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = tweet.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = tweet.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    
    reply_cnt = tweet.find_element_by_xpath('.//div[@data-testid="reply"]').text
    if(reply_cnt == ''):
        reply_cnt = 0
    retweet_cnt = tweet.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    if(retweet_cnt == ''):
        retweet_cnt = 0
    like_cnt = tweet.find_element_by_xpath('.//div[@data-testid="like"]').text
    if(like_cnt == ''):
        like_cnt = 0

    tweets = (username, handle, postdate, text, like_cnt, retweet_cnt,reply_cnt)
    return tweets    

In [29]:
for tweet in tweets:
    print(get_tweet_data(tweet))

NameError: name 'tweets' is not defined

In [52]:

# get all tweets on the page
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

i = 0
#SCRAPED 5 PAGES FOR TESTS
#while scrolling:
for i in range(5):
    page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        data.append(tweet)
            
    scroll_attempt = 0
    while True:
        #check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            #end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                time.sleep(2) #attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
driver.close()

In [53]:
print(data)
print(len(data))

[('Wheels Up Collective Marketing Group', '@wheelsupnow', '2022-01-26T14:10:04.000Z', "Wheels Up Collective Marketing Group\n@wheelsupnow\n·\n3hWhether you're an early stage startup just dipping your toes in marketing or an established enterprise looking for an outside perspective, we'd love to talk shop with you. Request your free marketing strategy assessment. https://buff.ly/3drzb4s  #marketingstrategy #marketingtips", 0, 0, 0), ('OilX', '@OilXs', '2022-01-26T12:39:31.000Z', 'OilX\n@OilXs\n·\n4hEarlier this week, we published our assessment of Nigeria´s highly anticipated Dangote Oil Refinery Company (DORC) startup for our clients. \n\nRequest a copy at research@oilx.co and learn more about OilX Research: https://hubs.li/Q012VYJq0\n\n#OOTT #downstream\n2\n5', '5', '2', 0), ("Paul O'Brien", '@seobrien', '2022-01-25T23:44:44.000Z', "Paul O'Brien\n@seobrien\n·\n17hProject for Hire Board? Upwork-like I suppose but more sophisticated and not based on skills. An RFP (request for proposal)

In [19]:
#saving to csv
import csv
with open('tweets.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['Author', 'Handle of Author', 'Date', 'Tweet', 'Likes', 'Retweets', 'Discussions']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)